# Interactive Point Cloud Visualization Demo

This notebook provides interactive 3D visualizations of point clouds and their transformations.


In [1]:
import os
import numpy as np
import plotly.graph_objects as go


In [2]:
def load_pointcloud(filepath):
    """Load point cloud from .npy file."""
    if not os.path.exists(filepath):
        return None
    return np.load(filepath)


def load_transformation(filepath):
    """Load 4x4 transformation matrix from .npy file."""
    if not os.path.exists(filepath):
        return None
    return np.load(filepath)


def apply_transformation(points, transform):
    """Apply 4x4 transformation matrix to points."""
    if points.shape[1] == 3:
        # Convert to homogeneous coordinates
        points_homo = np.hstack([points, np.ones((points.shape[0], 1))])
    else:
        points_homo = points
    
    transformed_homo = (transform @ points_homo.T).T
    return transformed_homo[:, :3]


def subsample_points(points, max_points=5000):
    """Subsample points if there are too many for visualization."""
    if points.shape[0] <= max_points:
        return points
    indices = np.random.choice(points.shape[0], max_points, replace=False)
    return points[indices]


def chain_transformations(run_name, start_frame, end_frame):
    """
    Chain transformation matrices from start_frame to end_frame.
    
    The transformation files are named transform_from_{t+1}_to_{t}.npy,
    which transforms FROM frame t+1 TO frame t. To go forward in time,
    we need to invert these transformations.
    
    Args:
        run_name: Name of the run (e.g., 'bird')
        start_frame: Starting frame number
        end_frame: Ending frame number (must be > start_frame)
    
    Returns:
        chained_transform: 4x4 transformation matrix from start_frame to end_frame
    """
    if end_frame <= start_frame:
        raise ValueError(f"end_frame ({end_frame}) must be greater than start_frame ({start_frame})")
    
    results_dir = os.path.join("results", run_name)
    registration_dir = os.path.join(results_dir, "6_registration")
    
    # Start with identity matrix
    chained_transform = np.eye(4)
    
    # Chain transformations from start_frame to end_frame
    for frame in range(start_frame, end_frame):
        # Load transformation from frame+1 to frame
        transform_path = os.path.join(registration_dir, f"transform_from_{frame+1}_to_{frame}.npy")
        
        if not os.path.exists(transform_path):
            raise FileNotFoundError(
                f"Missing transformation file: {transform_path}\n"
                f"Cannot chain transformations from frame {start_frame} to {end_frame}."
            )
        
        # Load the transformation (this goes FROM frame+1 TO frame)
        T_frame1_to_frame = load_transformation(transform_path)
        
        if T_frame1_to_frame is None:
            raise ValueError(f"Could not load transformation from {transform_path}")
        
        # Invert to get transformation FROM frame TO frame+1
        T_frame_to_frame1 = np.linalg.inv(T_frame1_to_frame)
        
        # Chain: T_start_to_end = T_start_to_start+1 @ T_start+1_to_start+2 @ ... @ T_end-1_to_end
        chained_transform = chained_transform @ T_frame_to_frame1
    
    return chained_transform


In [3]:
def visualize_pointclouds_interactive(run_name, frame_pair, max_points=5000, subsample_arrows=100):
    """
    Create interactive 3D visualization of point clouds and their transformations.
    
    Supports both consecutive frames (e.g., (8, 9)) and non-consecutive frames (e.g., (8, 17)).
    For non-consecutive frames, transformations are chained together.
    
    Args:
        run_name: Name of the run (e.g., 'bird')
        frame_pair: Tuple of (frame_t, frame_t1) to visualize. Can be consecutive or non-consecutive.
        max_points: Maximum number of points to display per point cloud
        subsample_arrows: Number of arrows to show (for transformation vectors)
    
    Returns:
        plotly.graph_objects.Figure: Interactive 3D plot
    """
    t, t_end = frame_pair
    
    if t_end <= t:
        raise ValueError(f"End frame ({t_end}) must be greater than start frame ({t})")
    
    # Setup paths
    results_dir = os.path.join("results", run_name)
    pointclouds_dir = os.path.join(results_dir, "5_pointclouds")
    registration_dir = os.path.join(results_dir, "6_registration")
    
    # Load point clouds
    pc_t_path = os.path.join(pointclouds_dir, f"pointcloud_{t:05d}.npy")
    pc_t_end_path = os.path.join(pointclouds_dir, f"pointcloud_{t_end:05d}.npy")
    
    print(f"Loading point clouds...")
    print(f"  Frame {t}: {pc_t_path}")
    print(f"  Frame {t_end}: {pc_t_end_path}")
    
    pc_t = load_pointcloud(pc_t_path)
    pc_t_end = load_pointcloud(pc_t_end_path)
    
    if pc_t is None:
        raise FileNotFoundError(f"Could not load point cloud at frame {t}")
    if pc_t_end is None:
        raise FileNotFoundError(f"Could not load point cloud at frame {t_end}")
    
    print(f"\nLoaded point clouds:")
    print(f"  Frame {t}: {pc_t.shape[0]} points")
    print(f"  Frame {t_end}: {pc_t_end.shape[0]} points")
    
    # Get transformation from t to t_end
    if t_end == t + 1:
        # Consecutive frames: use direct transformation
        print(t_end, t)
        transform_path = os.path.join(registration_dir, f"transform_from_{t_end}_to_{t}.npy")
        print(f"  Loading direct transformation: {transform_path}")
        
        transform_t_end_to_t = load_transformation(transform_path)
        if transform_t_end_to_t is None:
            raise FileNotFoundError(f"Could not load transformation matrix from {transform_path}")
        
        # Invert to get transformation from t to t_end
        transform = np.linalg.inv(transform_t_end_to_t)
        print(f"  Using direct transformation (inverted)")
    else:
        # Non-consecutive frames: chain transformations
        print(f"  Chaining transformations from frame {t} to frame {t_end}...")
        transform = chain_transformations(run_name, t, t_end)
        print(f"  Successfully chained {t_end - t} transformations")
    
    print(f"  Final transform matrix shape: {transform.shape}")
    
    # Apply transformation
    pc_t_end_transformed = apply_transformation(pc_t_end, transform)
    
    # Calculate statistics
    distances = np.linalg.norm(pc_t_end - pc_t_end_transformed, axis=1)
    mean_dist = np.mean(distances)
    std_dist = np.std(distances)
    max_dist = np.max(distances)
    min_dist = np.min(distances)
    
    print(f"\nTransformation Statistics:")
    print(f"  Mean distance: {mean_dist:.6f}")
    print(f"  Std distance: {std_dist:.6f}")
    print(f"  Min distance: {min_dist:.6f}")
    print(f"  Max distance: {max_dist:.6f}")
    
    # Subsample for visualization
    pc_t = subsample_points(pc_t, max_points)
    pc_t_end = subsample_points(pc_t_end, max_points)
    pc_t_end_transformed = subsample_points(pc_t_end_transformed, max_points)
    
    # Create interactive 3D plot
    fig = go.Figure()
    
    # Add point cloud at frame t (target)
    fig.add_trace(go.Scatter3d(
        x=pc_t[:, 0],
        y=pc_t[:, 1],
        z=pc_t[:, 2],
        mode='markers',
        marker=dict(
            size=3,
            color='blue',
            opacity=0.6,
        ),
        name=f'Frame {t} (target)',
    ))
    
    # Add point cloud at frame t_end (source, before transformation)
    fig.add_trace(go.Scatter3d(
        x=pc_t_end[:, 0],
        y=pc_t_end[:, 1],
        z=pc_t_end[:, 2],
        mode='markers',
        marker=dict(
            size=3,
            color='orange',
            opacity=0.6,
        ),
        name=f'Frame {t_end} (source, before transform)',
    ))
    
    # Add transformed point cloud at frame t_end
    fig.add_trace(go.Scatter3d(
        x=pc_t_end_transformed[:, 0],
        y=pc_t_end_transformed[:, 1],
        z=pc_t_end_transformed[:, 2],
        mode='markers',
        marker=dict(
            size=3,
            color='green',
            opacity=0.6,
        ),
        name=f'Frame {t_end} (transformed)',
    ))
    
    # Add arrows showing transformation vectors
    arrow_indices = np.random.choice(len(pc_t_end), min(subsample_arrows, len(pc_t_end)), replace=False)
    for idx in arrow_indices:
        fig.add_trace(go.Scatter3d(
            x=[pc_t_end[idx, 0], pc_t_end_transformed[idx, 0]],
            y=[pc_t_end[idx, 1], pc_t_end_transformed[idx, 1]],
            z=[pc_t_end[idx, 2], pc_t_end_transformed[idx, 2]],
            mode='lines+markers',
            line=dict(color='red', width=3),
            marker=dict(size=[4, 4], color=['red', 'red']),
            showlegend=False,
            hoverinfo='skip',
        ))
    
    # Update layout
    frame_info = f"{t}→{t_end}" if t_end == t + 1 else f"{t}→{t_end} (chained, {t_end - t} steps)"
    fig.update_layout(
        title=f'Interactive 3D Point Cloud Registration<br>{run_name} - Frames {frame_info}<br>Mean Distance: {mean_dist:.6f}',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='data',
        ),
        width=1000,
        height=800,
    )
    
    return fig


## Example Usage

### Consecutive Frames (t and t+1)

Visualize point clouds for consecutive frames:


You can adjust the parameters:
- `max_points`: Maximum number of points per point cloud (default: 5000)
- `subsample_arrows`: Number of transformation arrows to display (default: 100)

**Note:** For non-consecutive frames, the function will automatically chain all intermediate transformations. If any transformation in the chain is missing, an error will be raised.

Try different frame pairs or runs:


In [6]:
# Example: Visualize consecutive frames (e.g., 14 to 15)
fig = visualize_pointclouds_interactive(
    run_name='testo_testo',
    frame_pair=(41, 42),  # Consecutive frames
    max_points=100,  # Adjust based on your system's performance
    subsample_arrows=20  # Number of transformation arrows to show
)

# Display the interactive plot
fig.show()


Loading point clouds...
  Frame 41: results/testo_testo/5_pointclouds/pointcloud_00041.npy
  Frame 42: results/testo_testo/5_pointclouds/pointcloud_00042.npy

Loaded point clouds:
  Frame 41: 8087 points
  Frame 42: 8145 points
42 41
  Loading direct transformation: results/testo_testo/6_registration/transform_from_42_to_41.npy
  Using direct transformation (inverted)
  Final transform matrix shape: (4, 4)

Transformation Statistics:
  Mean distance: 0.004377
  Std distance: 0.000806
  Min distance: 0.001826
  Max distance: 0.014553


### Non-Consecutive Frames

Visualize point clouds across multiple frames. Transformations are automatically chained together:


In [ ]:
# Example: Visualize non-consecutive frames (e.g., 8 to 17)
# This will chain transformations: 8→9, 9→10, ..., 16→17
fig = visualize_pointclouds_interactive(
    run_name='walking_robot_dog',
    frame_pair=(1, 21),  # Non-consecutive frames - transformations will be chained
    max_points=1000,
    subsample_arrows=50
)

# Display the interactive plot
fig.show()
